In [6]:
data_path = 'Diseases/'
model_weights = 'disease_classification.h5'

batch_size = 32
valid_size = 16
color_mode = 'rgb'

width = 299
height = 299

target_size = (width, height)
input_shape = (width, height, 3)

zoom_range = 0.3
shear_range = 0.3
shift_range = 0.3
rotation_range = 30

dense_1 = 512
dense_2 = 256
dense_3 = 64
num_classes = 7

epochs = 50
rate = 0.2

verbose = 1

In [7]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import cv2 as cv
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.activations import relu
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
############################################################################################

def preprocessing_function(img):
    img = tf.keras.applications.xception.preprocess_input(img)
    return img

def image_data_generator():
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                                                rotation_range = rotation_range,
                                                                shear_range = shear_range,
                                                                zoom_range = zoom_range,
                                                                width_shift_range=shift_range,
                                                                height_shift_range=shift_range,
                                                                horizontal_flip = True,
                                                                preprocessing_function=preprocessing_function
                                                                )

    data_generator = train_datagen.flow_from_directory(
                                    data_path,
                                    target_size = target_size,
                                    color_mode = color_mode,
                                    batch_size = batch_size,
                                    class_mode = 'categorical',
                                    shuffle = True
                                    )

    return data_generator

In [8]:
data_generator = image_data_generator()
data_generator.class_indices

Found 454 images belonging to 7 classes.


{'Bovine Johne_s Disease': 0,
 'Foot _ Mouth Disease': 1,
 'Healthy  Cattle': 2,
 'Lumpy Skin Disease': 3,
 'Mastitis Disease': 4,
 'Milk Fever Disease': 5,
 'Unknown': 6}

In [9]:
class CattleDiseaseClassification(object):
    def __init__(self):
        self.data_generator = data_generator
        self.train_step = self.data_generator.samples // batch_size

        self.accuracy = tf.keras.metrics.CategoricalAccuracy()
        self.recall = tf.keras.metrics.Recall()
        self.precision = tf.keras.metrics.Precision()

        self.id2dis = {v:k for k, v in self.data_generator.class_indices.items()}

    def classifier(self, x):
        if not self.trainable:
            x = Dense(dense_1)(x)
            x = BatchNormalization()(x)
            x = relu(x)
            x = Dropout(rate)(x)

            x = Dense(dense_2)(x)
            x = BatchNormalization()(x)
            x = relu(x)
            x = Dropout(rate)(x)

        x = Dense(dense_3)(x)
        x = BatchNormalization()(x)
        x = relu(x)
        x = Dropout(rate)(x)
        return x

    def model_conversion(self, trainable):
        functional_model = tf.keras.applications.Xception(weights="imagenet")
        functional_model.trainable = trainable

        self.trainable = trainable

        inputs = functional_model.input

        x = functional_model.layers[-2].output
        x = self.classifier(x)
        outputs = Dense(num_classes, activation='softmax')(x)

        model = Model(
                inputs=inputs,
                outputs=outputs
                    )
                    
        self.model = model
        self.model.summary()

    def train(self):
        callback = tf.keras.callbacks.EarlyStopping(
                                                monitor='loss', 
                                                patience=5
                                                    )

        self.model.compile(
                          optimizer='Adam',
                          loss='categorical_crossentropy',
                          metrics=[
                                self.accuracy,
                                self.recall,
                                self.precision
                                  ]
                          )
        self.model.fit(
                    self.data_generator,
                    steps_per_epoch= self.train_step,
                    epochs=epochs,
                    verbose=verbose
                        )

    def save_model(self):
        self.model.save(model_weights)

    def load_model(self):
        self.model = load_model(model_weights)
        self.model.compile(
                          optimizer='Adam',
                          loss='categorical_crossentropy',
                          metrics=[
                                self.accuracy,
                                self.recall,
                                self.precision
                                  ]
                         )

    def predict(self, x):
        x = cv.resize(x, target_size)
        x = preprocessing_function(x)
        x = np.expand_dims(x, axis=0)
        P = self.model.predict(x)
        idx = np.argmax(P)
        return self.id2dis[idx]
        
    def process(self):
        if not os.path.exists(model_weights):
            self.model_conversion(False)
            self.train()
            self.save_model()
        else:
            self.load_model()

In [10]:
model = CattleDiseaseClassification()
model.process()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                           

 block4_sepconv1_bn (BatchNorma  (None, 37, 37, 728)  2912       ['block4_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block4_sepconv2_act (Activatio  (None, 37, 37, 728)  0          ['block4_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 37, 37, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block4_sepconv2_bn (BatchNorma  (None, 37, 37, 728)  2912       ['block4_sepconv2[0][0]']        
 lization)

 n)                                                                                               
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, 19, 19, 728)  536536     ['block7_sepconv1_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv1_bn (BatchNorma  (None, 19, 19, 728)  2912       ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_sepconv2_act (Activatio  (None, 19, 19, 728)  0          ['block7_sepconv1_bn[0][0]']     
 n)                                                                                               
          

 block9_sepconv3_bn (BatchNorma  (None, 19, 19, 728)  2912       ['block9_sepconv3[0][0]']        
 lization)                                                                                        
                                                                                                  
 add_19 (Add)                   (None, 19, 19, 728)  0           ['block9_sepconv3_bn[0][0]',     
                                                                  'add_18[0][0]']                 
                                                                                                  
 block10_sepconv1_act (Activati  (None, 19, 19, 728)  0          ['add_19[0][0]']                 
 on)                                                                                              
                                                                                                  
 block10_sepconv1 (SeparableCon  (None, 19, 19, 728)  536536     ['block10_sepconv1_act[0][0]']   
 v2D)     

                                                                                                  
 block12_sepconv3_act (Activati  (None, 19, 19, 728)  0          ['block12_sepconv2_bn[0][0]']    
 on)                                                                                              
                                                                                                  
 block12_sepconv3 (SeparableCon  (None, 19, 19, 728)  536536     ['block12_sepconv3_act[0][0]']   
 v2D)                                                                                             
                                                                                                  
 block12_sepconv3_bn (BatchNorm  (None, 19, 19, 728)  2912       ['block12_sepconv3[0][0]']       
 alization)                                                                                       
                                                                                                  
 add_22 (A

                                                                                                  
 batch_normalization_13 (BatchN  (None, 64)          256         ['dense_6[0][0]']                
 ormalization)                                                                                    
                                                                                                  
 tf.nn.relu_5 (TFOpLambda)      (None, 64)           0           ['batch_normalization_13[0][0]'] 
                                                                                                  
 dropout_5 (Dropout)            (None, 64)           0           ['tf.nn.relu_5[0][0]']           
                                                                                                  
 dense_7 (Dense)                (None, 7)            455         ['dropout_5[0][0]']              
                                                                                                  
Total para

14/14 [==============================] - 41s 3s/step - loss: 0.1821 - categorical_accuracy: 0.9384 - recall_1: 0.9289 - precision_1: 0.9515
Epoch 47/50
14/14 [==============================] - 41s 3s/step - loss: 0.1375 - categorical_accuracy: 0.9645 - recall_1: 0.9526 - precision_1: 0.9757
Epoch 48/50
14/14 [==============================] - 41s 3s/step - loss: 0.1563 - categorical_accuracy: 0.9502 - recall_1: 0.9479 - precision_1: 0.9592
Epoch 49/50
14/14 [==============================] - 41s 3s/step - loss: 0.1767 - categorical_accuracy: 0.9431 - recall_1: 0.9313 - precision_1: 0.9516
Epoch 50/50
14/14 [==============================] - 43s 3s/step - loss: 0.1719 - categorical_accuracy: 0.9487 - recall_1: 0.9308 - precision_1: 0.9630


# **Inference**

In [11]:
img_path = 'Diseases/Foot _ Mouth Disease/FM1_256x256.jpg'
img = cv.imread(img_path)
P = model.predict(img)
P

'Foot _ Mouth Disease'

In [12]:
img_path = 'Diseases/Healthy  Cattle/NC18_256x256.jpg'
img = cv.imread(img_path)
P = model.predict(img)
P

'Healthy  Cattle'

In [13]:
img_path = 'Diseases/Lumpy Skin Disease/Lumpy7_256x256.jpg'
img = cv.imread(img_path)
P = model.predict(img)
P

'Lumpy Skin Disease'

In [14]:
img_path = 'Diseases/Mastitis Disease/Mastitis33_256x256.jpg'
img = cv.imread(img_path)
P = model.predict(img)
P

'Mastitis Disease'

In [15]:
img_path = 'Diseases/Milk Fever Disease/Milk19_256x256.jpg'
img = cv.imread(img_path)
P = model.predict(img)
P

'Milk Fever Disease'

In [16]:
img_path = 'Diseases/Unknown/6ToYKVm.jpg'
img = cv.imread(img_path)
P = model.predict(img)
P

'Unknown'

In [17]:
img_path = 'Diseases/Bovine Johne_s Disease/BJD16_256x256.jpg'
img = cv.imread(img_path)
P = model.predict(img)
P

'Bovine Johne_s Disease'